<a href="https://colab.research.google.com/github/vohai2003/Pythoncode/blob/main/ANOVA_doan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lấy dữ liệu và load thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Dataset/"

Mounted at /content/drive


In [ ]:
import numpy
import pandas
import scipy.stats
import matplotlib

In [ ]:
df = pandas.read_csv(path+"Diet_R.csv")
df

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,,41,171,60,2,60.0
1,26,,32,174,103,2,103.0
2,1,0,22,159,58,1,54.2
3,2,0,46,192,60,1,54.0
4,3,0,55,170,64,1,63.3
...,...,...,...,...,...,...,...
73,74,1,35,183,83,3,80.2
74,75,1,49,177,84,3,79.9
75,76,1,28,164,85,3,79.7
76,77,1,40,167,87,3,77.8


Xử lý dữ liệu cho phù hợp với câu hỏi nghiên cứu: Sự khác biệt về hiệu quả giữa các chế độ ăn kiêng

In [ ]:
method1 = df[df.Diet == 1]
method2 = df[df.Diet == 2]
method3 = df[df.Diet == 3]
refined = pandas.DataFrame()
refined.insert(0,"X1",(method1["weight6weeks"]-method1["pre.weight"]))
refined = refined.reset_index(drop=True)
refined.insert(1,"X2",pandas.Series((method2["weight6weeks"]-method2["pre.weight"]).values))
refined = refined.reset_index(drop=True)
refined.insert(2,"X3",pandas.Series((method3["weight6weeks"]-method3["pre.weight"]).values))
refined = refined.reset_index(drop=True)

In [ ]:
refined

,X1,X2,X3
0,-3.8,0.0,-7.0
1,-6.0,0.0,-5.6
2,-0.7,2.1,-3.4
3,-2.9,-2.0,-6.8
4,-2.8,-1.7,-7.8
5,-2.0,-4.3,-5.4
6,-2.0,-7.0,-6.8
7,-8.5,-0.6,-7.2
8,-1.9,-2.7,-7.0
9,-3.1,-3.6,-7.3


Giả thuyết của các kiểm định: \
Bartlett: \
$H_0:$ Dữ liệu theo phân phối chuẩn \
$H_a:$ Dữ liệu không theo phân phối chuẩn 
\

Shapiro-Wilk: \
$H_0:$ Hai nhóm có phương sai bằng nhau \
$H_a:$ Hai nhóm có phương sai khác nhau \
hay: \
$H_0: σ_1 = σ_2$ \
$H_a: σ_1 ≠ σ_2$

Hàm kiểm định Bartlett và Shapiro-Wilk

In [ ]:
alpha = 0.05
#Hàm kiểm định phân phối chuẩn cho toàn bộ data trong dataframe
def norm_dist(dt:pandas.DataFrame):
  result = True
  for column in dt.columns:
    st , p = scipy.stats.shapiro(dt[column])
    if p < alpha:
      result = False
      break
  return result
#Hàm kiểm định phương sai cho toàn dataframe
def diff_deviation(dt:pandas.DataFrame):
  result = True
  for column in dt.columns:
    for column2 in dt.columns:
      if column != column2:
        st , p = scipy.stats.bartlett(dt[column],dt[column2])
        if p < alpha:
          result = False
  return result

In [ ]:
print(norm_dist(refined))
print(diff_deviation(refined))

True
True


Giả thuyết của kiểm định ANOVA: \
$H_0:$ Trung bình của các nhóm bằng nhau \
$H_a:$ Trung bình của các nhóm khác nhau \
hay: \
$H_0: X_1 = X_2 = X_3$ \
$H_a:$ Ít nhất 1 cặp giá trị trung bình khác biệt \

In [ ]:
ST, p = scipy.stats.f_oneway(refined["X1"],refined["X2"],refined["X3"])
print("p =",p)

p = 0.009001196939385125
